In [ ]:

import streamlit as st
import tensorflow as tf
from PIL import Image
import numpy as np
# Import fungsi preprocessing spesifik untuk MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# --- Konfigurasi Awal (dari kode Anda) ---
# Daftar label kelas daun (sudah benar)
class_names = [
    'Ancak', 'AwarAwar', 'Beringin', 'Cemara', 'CempakaKuning',
    'Ceremai', 'Dapdap', 'Delima', 'Gadung', 'JambuAir',
    'JambuBiji', 'JarakPagar', 'JerukLimau', 'JerukSitrun', 'Juwet',
    'KacaPiring', 'Kamboja', 'Kasimbukan', 'Kecubung', 'Kelor',
    'Keluak', 'KembangSepatu', 'Kepundung', 'KetimunGantung', 'Kopi',
    'Lantana', 'Legundi', 'ManggaAmplem', 'Manggis', 'Mengkudu',
    'Merica', 'Nanas', 'PakisSayur', 'Pala', 'Pare',
    'Pepaya', 'Pule', 'Salam', 'Semanggi', 'Sembung',
    'Sirih', 'Sirsak', 'Sisih', 'Srikaya', 'Suren',
    'Talas', 'Teleng', 'TerongDuri', 'UyahUyah', 'Wani'
]

khasiat_daun = {
    'Ancak': 'Menurunkan demam dan membantu mengatasi gangguan pencernaan.',
    'AwarAwar': 'Mengobati luka, mengurangi peradangan, dan mengatasi infeksi kulit.',
    'Beringin': 'Membantu mengatasi diare, bersifat antiseptik, dan menurunkan demam.',
    'Cemara': 'Meredakan batuk, menghangatkan tubuh, dan bersifat antimikroba.',
    'CempakaKuning': 'Mengurangi peradangan, meredakan nyeri, dan menenangkan pikiran.',
    'Ceremai': 'Menurunkan kadar gula darah, sebagai antioksidan, dan meredakan batuk.',
    'Dapdap': 'Mengobati luka, mengurangi nyeri, dan menenangkan saraf.',
    'Delima': 'Kaya antioksidan, menjaga kesehatan jantung, dan menurunkan tekanan darah.',
    'Gadung': 'Mengurangi nyeri rematik dan peradangan setelah diolah dengan benar.',
    'JambuAir': 'Membantu mengontrol gula darah dan menjaga sistem pencernaan.',
    'JambuBiji': 'Mengatasi diare, bersifat antibakteri, dan menurunkan kolesterol.',
    'JarakPagar': 'Mengobati luka, bersifat antiinflamasi, dan antibakteri.',
    'JerukLimau': 'Meredakan batuk dan menyegarkan pernapasan.',
    'JerukSitrun': 'Meningkatkan daya tahan tubuh, antioksidan, dan membantu detoksifikasi.',
    'Juwet': 'Menurunkan gula darah dan membantu mengatasi diare.',
    'KacaPiring': 'Mengurangi peradangan, menenangkan tubuh, dan membantu mengatasi insomnia.',
    'Kamboja': 'Meredakan nyeri sendi dan bersifat antiinflamasi.',
    'Kasimbukan': 'Mengatasi sakit perut dan bersifat antimikroba.',
    'Kecubung': 'Digunakan sebagai pereda nyeri dan obat asma tradisional (penggunaan terbatas).',
    'Kelor': 'Meningkatkan imunitas, menurunkan gula darah, dan kaya antioksidan.',
    'Keluak': 'Membantu penyembuhan luka dan bersifat antibakteri.',
    'KembangSepatu': 'Menyuburkan rambut dan membantu meredakan batuk.',
    'Kepundung': 'Menurunkan demam dan melancarkan pencernaan.',
    'KetimunGantung': 'Menurunkan tekanan darah dan membantu detoksifikasi tubuh.',
    'Kopi': 'Mengandung antioksidan dan membantu meningkatkan metabolisme.',
    'Lantana': 'Bersifat antiseptik dan membantu mengobati luka.',
    'Legundi': 'Meredakan batuk dan mengurangi peradangan.',
    'ManggaAmplem': 'Sebagai antioksidan dan membantu menjaga pencernaan.',
    'Manggis': 'Mengandung antioksidan tinggi dan bersifat antikanker.',
    'Mengkudu': 'Menurunkan tekanan darah dan meningkatkan daya tahan tubuh.',
    'Merica': 'Melancarkan pencernaan dan bersifat antibakteri.',
    'Nanas': 'Mengurangi peradangan dan membantu pencernaan.',
    'PakisSayur': 'Menyehatkan sistem pencernaan dan kaya antioksidan.',
    'Pala': 'Menenangkan saraf dan membantu mengatasi insomnia.',
    'Pare': 'Menurunkan kadar gula darah dan membantu pengobatan diabetes.',
    'Pepaya': 'Melancarkan pencernaan dan membantu meningkatkan trombosit.',
    'Pule': 'Menurunkan demam dan digunakan sebagai obat antimalaria tradisional.',
    'Salam': 'Menurunkan kolesterol dan membantu mengontrol gula darah.',
    'Semanggi': 'Melancarkan peredaran darah dan meningkatkan stamina.',
    'Sembung': 'Mengatasi masuk angin dan bersifat antiinflamasi.',
    'Sirih': 'Sebagai antiseptik alami dan menjaga kesehatan mulut.',
    'Sirsak': 'Dipercaya memiliki sifat antikanker dan antibakteri.',
    'Sisih': 'Digunakan sebagai obat luka tradisional.',
    'Srikaya': 'Mengatasi peradangan dan digunakan untuk menghilangkan kutu.',
    'Suren': 'Bersifat antimikroba dan membantu mengatasi infeksi kulit.',
    'Talas': 'Menjaga kesehatan pencernaan dan sebagai sumber antioksidan.',
    'Teleng': 'Kaya antioksidan dan menyehatkan mata.',
    'TerongDuri': 'Meredakan nyeri dan peradangan.',
    'UyahUyah': 'Menurunkan tekanan darah dan membantu melancarkan ASI.',
    'Wani': 'Sebagai antioksidan dan meningkatkan daya tahan tubuh.'
}

# --- Peningkatan Performa dan Akurasi ---

# 1. Gunakan cache untuk memuat model. Model hanya akan di-load sekali.
@st.cache_resource
def load_model():
    """Memuat model machine learning dari file .h5"""
    # Sesuaikan 'mobilenetv2_model.h5' dengan path/nama file model Anda
    model = tf.keras.models.load_model('mobilenetv2_model.h5')
    return model

# 2. Fungsi untuk preprocessing dan prediksi
def preprocess_and_predict(model, image):
    """
    Melakukan pra-pemrosesan gambar dan mengembalikan prediksi,
    nama kelas, dan skor keyakinan.
    """
    # Pastikan gambar dalam mode RGB
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Resize gambar ke ukuran yang diharapkan oleh model
    image = image.resize((224, 224))

    # Ubah gambar menjadi array numpy
    image_array = np.array(image)

    # Tambahkan dimensi batch
    image_array = np.expand_dims(image_array, axis=0)

    # Gunakan preprocess_input yang sesuai untuk MobileNetV2
    processed_image = preprocess_input(image_array)

    # Lakukan prediksi
    prediction = model.predict(processed_image)

    # Dapatkan indeks, nama, dan skor keyakinan
    predicted_class_index = np.argmax(prediction)
    predicted_class_name = class_names[predicted_class_index]
    confidence_score = np.max(prediction) * 100

    return predicted_class_name, confidence_score

# Muat model menggunakan fungsi yang sudah di-cache
model = load_model()

# --- UI Layout (dari kode Anda) ---
st.set_page_config(page_title="Klasifikasi Daun Herbal", layout="centered")

st.markdown(
    "<h3 style='text-align: center;'>"
    "<span style='color:#0abf53;'>CARI</span> TANAMAN HERBAL<br>"
    "<span style='color:#1abc9c;'>DENGAN</span> SATU KLIK!!"
    "</h3>", unsafe_allow_html=True)

st.markdown("---")

uploaded_file = st.file_uploader("Unggah gambar daun", type=["jpg", "jpeg", "png"])

if uploaded_file is not None and model is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Gambar Daun yang Diunggah", use_column_width=True)

    with st.spinner("Menganalisis gambar..."):
        # Panggil fungsi untuk prediksi
        predicted_class, confidence = preprocess_and_predict(model, image)
        manfaat = khasiat_daun.get(predicted_class, "Khasiat tidak ditemukan.")

    st.markdown("---")
    # Tampilkan hasil dengan tambahan skor keyakinan
    st.markdown(
        f"<p style='text-align:center; font-size:24px; color:#0abf53;'>"
        f"<strong>{predicted_class}</strong></p>",
        unsafe_allow_html=True
    )
    st.markdown(
        f"<p style='text-align:center; font-size:16px;'>"
        f"Tingkat Keyakinan: <strong>{confidence:.2f}%</strong></p>",
        unsafe_allow_html=True
    )
    st.markdown(
        f"<p style='text-align:center; font-size:18px; color:#444; border-left: 4px solid #0abf53; padding-left: 10px;'>"
        f"<strong>Khasiat:</strong> {manfaat}</p>",
        unsafe_allow_html=True
    )
elif model is None:
    st.error("Gagal memuat model. Mohon periksa file model dan coba lagi.")


# Footer Icons (dari kode Anda)
st.markdown("<br><br><br>", unsafe_allow_html=True)
st.markdown("---")
col1, col2, col3, col4, col5 = st.columns(5)
with col2:
    st.markdown('<div style="text-align:center;"><img src="https://img.icons8.com/ios/50/00e676/camera--v1.png" width="40"/></div>', unsafe_allow_html=True)
with col4:
    st.markdown('<div style="text-align:center;"><img src="https://img.icons8.com/ios/50/00e676/image.png" width="40"/></div>', unsafe_allow_html=True)